# API URL

In [1]:
base_url = "http://localhost:8080/"

# Example usage with text

In [ ]:
import requests
import json

# URL bạn nhận được sau khi triển khai Cloud Run
api_url = base_url + "generate"


# Prompt bạn muốn gửi
user_prompt = """Problem:\n                Julia was preparing for a dinner party at her house, where she intended to serve stew.  She noticed that she was out of plastic spoons, so she bought a new package of spoons.  Later, her husband also bought a package of 5 new spoons and gave them to Julia.  While Julia was making the stew, she used three of the spoons to sample her stew.  Later, when she went to set the table, she had a total of 12 spoons.  How many spoons were in the package that Julia bought?\n\n                Ground truth solution:\n                The total number of spoons from Julia and her husband was 12+3=15 spoons.\nSince the husband bought a package of five spoons, then Julia's package contained 15-5=10 spoons.\n 10\n\n                Dialogue history:\n                Teacher: Hi Mariana, please talk me through your solution\nStudent: Sure. I started by letting x be the number of spoons Julia bought. Then I added 5 to x to get the total number of spoons. Next, I subtracted 3 from the total number of spoons to get the number of spoons left. Finally, I set up an equation and solved for x, which was 4. So Julia bought a package of 4 spoons.\nTeacher: Okay -  you've overcomplicated this. Let's start again. We know Julia has 12 spoons when setting the table. We also know she used three whilst sampling her stew. How many is that altogether?\nStudent: 12 - 3 = 9. So Julia had 9 spoons left when setting the table.\nTeacher: No, you need to add as she still has the spoons, 3 have just been used\nStudent: Oh, right. 12 + 3 = 15. So Julia had 15 spoons altogether. And since her husband gave her 5 spoons, she must have bought 10 spoons.\n\n                Generate the next teacher response:"""

# Dữ liệu gửi đi dưới dạng JSON
payload = {"prompt": user_prompt, "language": "Tiếng Việt"}

# Gửi yêu cầu POST
response = requests.post(api_url, json=payload)

if response.status_code == 200:
    # In kết quả
    result = response.json()
    print(result.get("response"))
else:
    print("Error:", response.status_code, response.text)

# Example usage stream

In [ ]:
import requests
import json


api_url = base_url + "/generate/stream"

# Prompt bạn muốn gửi
user_prompt = """Problem:\n                Julia was preparing for a dinner party at her house, where she intended to serve stew.  She noticed that she was out of plastic spoons, so she bought a new package of spoons.  Later, her husband also bought a package of 5 new spoons and gave them to Julia.  While Julia was making the stew, she used three of the spoons to sample her stew.  Later, when she went to set the table, she had a total of 12 spoons.  How many spoons were in the package that Julia bought?\n\n                Ground truth solution:\n                The total number of spoons from Julia and her husband was 12+3=15 spoons.\nSince the husband bought a package of five spoons, then Julia's package contained 15-5=10 spoons.\n 10\n\n                Dialogue history:\n                Teacher: Hi Mariana, please talk me through your solution\nStudent: Sure. I started by letting x be the number of spoons Julia bought. Then I added 5 to x to get the total number of spoons. Next, I subtracted 3 from the total number of spoons to get the number of spoons left. Finally, I set up an equation and solved for x, which was 4. So Julia bought a package of 4 spoons.\nTeacher: Okay -  you've overcomplicated this. Let's start again. We know Julia has 12 spoons when setting the table. We also know she used three whilst sampling her stew. How many is that altogether?\nStudent: 12 - 3 = 9. So Julia had 9 spoons left when setting the table.\nTeacher: No, you need to add as she still has the spoons, 3 have just been used\nStudent: Oh, right. 12 + 3 = 15. So Julia had 15 spoons altogether. And since her husband gave her 5 spoons, she must have bought 10 spoons.\n\n                Generate the next teacher response:"""

# Dữ liệu gửi đi dưới dạng JSON
payload = {"prompt": user_prompt, "language": "Vietnamese"}

# Headers cho streaming request
headers = {"Content-Type": "application/json", "Accept": "text/event-stream"}

response = requests.post(api_url, json=payload, headers=headers, stream=True)

full_response = ""

# Đọc từng chunk từ response
if response.status_code == 200:
    for line in response.iter_lines(decode_unicode=True):
        if line and line.startswith("data: "):
            # Server-Sent Events format: "data: {json_data}"
            json_data = line[6:]  # Bỏ "data: "
            data = json.loads(json_data)
            print(f"{data.get('text')}")

# Example usage text + image

In [ ]:
import requests
import json
import os

# URL bạn nhận được sau khi triển khai Cloud Run
api_url = base_url + "generate/image"

image_path = "image.png"
file_name = os.path.basename(image_path)

# Prompt bạn muốn gửi
user_prompt = r"""
Giải bài sau:

"""

# Đọc file ảnh trước khi đóng
with open(image_path, "rb") as image_file:
    image_data = image_file.read()

# Tạo files dictionary với data đã đọc
files_to_upload = {"image": (file_name, image_data, "image/png")}

# Tạo form data (không dùng json khi có files)
form_data = {"prompt": user_prompt, "language": "vi"}

# Gửi yêu cầu POST với form data và files
response = requests.post(api_url, data=form_data, files=files_to_upload)

if response.status_code == 200:
    # In kết quả
    result = response.json()
    print("Response:", result.get("response"))
    if "image_info" in result:
        print("\nImage info:", result.get("image_info"))
else:
    print("Error:", response.status_code)
    print("Response text:", response.text)